# CRAW LEAKING APK INFO

In [143]:
import os
import time
import glob
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
import requests
import pandas as pd
import os
import urllib.parse
import threading
from time import sleep
from threading import Thread
import glob
import shutil

In [144]:
# Static variable
apk_csv_file = "C:\\Users\\ASUS\\anaconda3\\metadata-gdpr-client\\leaking_list.csv"

In [145]:
def exportResultCSV(df,apk_csv_file):
    # 3.2 Export
    df.to_csv(apk_csv_file,index=False)
    export = pd.read_csv(apk_csv_file,low_memory=False)
    export

In [146]:
# 0. Read CSV file
df = pd.read_csv(apk_csv_file,low_memory=False)
pkg_name_arr = df['pkg_name'].drop_duplicates().tolist()
print(len(pkg_name_arr))
print(pkg_name_arr)

617
['com.google.android.apps.docs', 'com.vsco.cam', 'com.prequel.app', 'com.adobe.lrmobile', 'com.linecorp.b612.android', 'com.niksoftware.snapseed', 'photo.editor.polarr', 'com.bigwinepot.nwdn.international', 'net.sourceforge.opencamera', 'com.photogrid.collage.videomaker', 'com.adobe.psmobile', 'com.alensw.PicFolder', 'com.mt.mtxx.mtxx', 'photo.editor.photoeditor.photoeditorpro', 'com.atomicadd.fotos', 'vn.eraser.background.removebg', 'com.sec.android.gallery3d', 'com.imaginstudio.imagetools.pixellab', 'com.joeware.android.gpulumera', 'com.commsource.beautyplus', 'com.cyworld.camera', 'com.sec.android.app.camera', 'com.lightricks.quickshot', 'com.venticake.retrica', 'com.kii.safe', 'com.cyberlink.youperfect', 'com.cyberlink.youcammakeup', 'com.gpsmapcamera.geotagginglocationonphoto', 'com.campmobile.snow', 'com.simplemobiletools.gallery', 'com.flipcam', 'air.com.befunky.BeFunkyPhotoEditor', 'com.roidapp.photogrid', 'kr.co.ladybugs.fourto', 'filter.camera.snap.photo.video.panorama', 

In [147]:
# 1. Get data
try:
    # 1.1 Setup selenium
    chromedriver = 'chromedriver.exe'
    os.environ["webdriver.chrome.driver"] = chromedriver
    chrome_options = Options()
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_argument("--disable-popup-blocking")
    chrome_options.add_argument('--ignore-ssl-errors=yes')
    chrome_options.add_argument('--ignore-certificate-errors')
    chrome_options.add_argument('--allow-running-insecure-content')
    chrome_options.add_argument('--no-proxy-server')
    # open web in background
    #chrome_options.add_argument('--headless=new')
    # open web in background
    i = 31
    for i in range(31,len(pkg_name_arr)):
        print("----------------------Loop i = "+str(i)+"----------------------")
        # 1.2. Load web page
        web = webdriver.Chrome(options=chrome_options)
        web.maximize_window()
        web.get("https://apkcombo.com")
        time.sleep(5)
        # 1.3 Agree privacy
        try:
            privacy_agree=web.find_element(By.CSS_SELECTOR,"button[class='fc-button fc-cta-consent fc-primary-button']")
            privacy_agree.click()
            time.sleep(5)
        except NoSuchElementException:
            print("Element Agree privacy does not exist")
        # 1.4. Find textbox and input apk name
        apk_name_input=web.find_element(By.NAME, 'q')
        print("pkg_name: "+pkg_name_arr[i])
        search_apk=apk_name_input.send_keys(pkg_name_arr[i])
        time.sleep(5)
        search=web.find_element(By.CSS_SELECTOR,"button[class='button button-search is-link']")
        search.click()
        time.sleep(5)
        #1.5 Get purpose
        try:
            apk_purpose_general = web.find_element(By.CSS_SELECTOR,"div[class='app_name']").text
            print("apk_purpose_general: "+apk_purpose_general)
        except NoSuchElementException:
            print("App does not exist")
            continue
        apk_purpose_detail = web.find_element(By.CSS_SELECTOR,"h2[class='mt-14 mb-14 short-description']").text
        print("apk_purpose_detail: "+apk_purpose_detail)
        # 1.6 Get APK info
        apk_infos=web.find_elements(By.CSS_SELECTOR,"div[class='value']")
        #print(len(apk_infos))
        apk_info_arr = []
        for j in range(len(apk_infos)):
            apk_info=web.find_elements(By.CSS_SELECTOR,"div[class='value']")[j].text
            apk_info_arr.append(apk_info)
        print(apk_info_arr) 
        developer = apk_info_arr[2]
        category = apk_info_arr[3]
        install = (apk_info_arr[5]).replace(",", "").replace("+", "")
        print("developer: "+developer)
        print("category: "+category)
        print("install: "+install)
        # 1.7 Update csv file
        df.loc[df['pkg_name'] == pkg_name_arr[i], 'installs'] = install
        df.loc[df['pkg_name'] == pkg_name_arr[i], 'developer'] = developer
        df.loc[df['pkg_name'] == pkg_name_arr[i], 'category'] = category
        df.loc[df['pkg_name'] == pkg_name_arr[i], 'purpose'] = apk_purpose_general
        df.loc[df['pkg_name'] == pkg_name_arr[i], 'detail'] = apk_purpose_detail
        web.close()
        time.sleep(2)
except Exception as e:
    print(f"Error occurred: {str(e)}")
    exportResultCSV(df,apk_csv_file)
    web.close()
    time.sleep(2)

----------------------Loop i = 31----------------------
pkg_name: air.com.befunky.BeFunkyPhotoEditor
App does not exist
----------------------Loop i = 32----------------------
pkg_name: com.roidapp.photogrid
apk_purpose_general: PhotoGrid: Video & Pic Collage Maker, Photo Editor
apk_purpose_detail: Camera app for Video & Photo Collage, Video maker with Music, Text& Sticker edit
['8.62 (86200020)', 'Aug 17, 2023', 'Cheetah Mobile (Photo Editor)', 'Photography', 'com.roidapp.photogrid', '100,000,000+']
developer: Cheetah Mobile (Photo Editor)
category: Photography
install: 100000000
----------------------Loop i = 33----------------------
pkg_name: kr.co.ladybugs.fourto
apk_purpose_general: FOTO Gallery
apk_purpose_detail: Fix the malfunction of functions such as moving and deleting photos.
['4.00.29 (129)', 'Jul 20, 2021', 'Gion Lab', 'Photography', 'kr.co.ladybugs.fourto', '5,000,000+']
developer: Gion Lab
category: Photography
install: 5000000
----------------------Loop i = 34---------

----------------------Loop i = 52----------------------
pkg_name: photography.redgallery.tools
apk_purpose_general: IPhone Gallery
apk_purpose_detail: IPhone Gallery is an picture manager for viewing your photos, prime videos
['1.4 (14)', 'Aug 16, 2021', 'blackcaps', 'Photography', 'photography.redgallery.tools', '50,000+']
developer: blackcaps
category: Photography
install: 50000
----------------------Loop i = 53----------------------
pkg_name: com.cyberlink.photodirector
apk_purpose_general: PhotoDirector: AI Photo Editor
apk_purpose_detail: Edit and animate pictures with powerful AI and creative tools!
['18.3.5 (90180350)', 'Aug 18, 2023', 'Cyberlink Corp', 'Photography', 'com.cyberlink.photodirector', '50,000,000+']
developer: Cyberlink Corp
category: Photography
install: 50000000
----------------------Loop i = 54----------------------
pkg_name: com.sergeyotro.sharpsquare
apk_purpose_general: #SquareDroid: Full Size Photo for Instagram and DP
apk_purpose_detail: Fit Instagram photo

pkg_name: com.hdcamera.hdcameraiphone7
apk_purpose_general: HD Camera For iphone7 Plus
apk_purpose_detail: the best camera app for your smartphone
['30.04.24 (105)', 'Feb 4, 2018', 'exelart', 'Photography', 'com.hdcamera.hdcameraiphone7', '100+']
developer: exelart
category: Photography
install: 100
----------------------Loop i = 75----------------------
pkg_name: com.snowcorp.soda.android
apk_purpose_general: SODA Camera - SODA - Natural Beauty Camera
apk_purpose_detail: the easy and simple beauty camera
['7.1.2 (30701002)', 'Jul 11, 2023', 'SNOW Corporation', 'Photography', 'com.snowcorp.soda.android', '10,000,000+']
developer: SNOW Corporation
category: Photography
install: 10000000
----------------------Loop i = 76----------------------
pkg_name: com.theronrogers.vaultyfree
apk_purpose_general: Vaulty : Hide Pictures Videos
apk_purpose_detail: Safe, Private and Secret Hidden Photo Vault app. Hide photos & video in Gallery.
['23.07.40 release (460740)', 'Feb 23, 2023', 'Squid Tooth 

----------------------Loop i = 96----------------------
pkg_name: us.koller.cameraroll
apk_purpose_general: Camera Roll - Gallery
apk_purpose_detail: Camera Roll is a simple and beautiful Gallery App.
['v1.0.6 (36)', 'Mar 23, 2018', 'Lukas Koller', 'Photography', 'us.koller.cameraroll', '500,000+']
developer: Lukas Koller
category: Photography
install: 500000
----------------------Loop i = 97----------------------
pkg_name: com.PicFree
App does not exist
----------------------Loop i = 98----------------------
pkg_name: com.camera.one.s10.camera
apk_purpose_general: One S10 Camera -Galaxy S10 cam
apk_purpose_detail: One S10 Camera is Galaxy S10 camera with amazing features, filters, AR stickers
['6.0 (54)', 'Aug 14, 2023', 'Model X Apps', 'Photography', 'com.camera.one.s10.camera', '10,000,000+']
developer: Model X Apps
category: Photography
install: 10000000
----------------------Loop i = 99----------------------
pkg_name: com.wombatica.facewarp
apk_purpose_general: Face Warp
apk_purpo

----------------------Loop i = 118----------------------
pkg_name: com.camera.lmc84
apk_purpose_general: LMC8.4 - Google Camera
apk_purpose_detail: LMC Google Camera Port by Hasli
['8.4_R18 (64764891)', 'Jun 19, 2023', 'Hasli', 'Photography', 'com.camera.lmc84', '100,000+']
developer: Hasli
category: Photography
install: 100000
----------------------Loop i = 119----------------------
pkg_name: de.ultrareface.appreface
apk_purpose_general: Reface Ultra
apk_purpose_detail: Cool & Fun Photo Effects
['1.0 (1)', 'Aug 30, 2021', 'Kramar Ro.', 'Photography', 'de.ultrareface.appreface', '50,000+']
developer: Kramar Ro.
category: Photography
install: 50000
----------------------Loop i = 120----------------------
pkg_name: app.galleryx
apk_purpose_general: 1Gallery:Photo Gallery & Vault
apk_purpose_detail: A beautiful photo gallery - Besides, it supports hidden and encrypted pictures.
['1.0.7-7.130123 (92)', 'Jan 13, 2023', 'todayweather.co', 'Photography', 'app.galleryx', '100,000+']
developer:

...

----------------------Loop i = 526----------------------
pkg_name: com.goodwy.gallery
apk_purpose_general: Right Gallery
apk_purpose_detail: Photo editor & gallery. Crop pictures & edit photos.
['4.0.2 (402)', 'Mar 30, 2023', 'Goodwy', 'Photography', 'com.goodwy.gallery', '10,000+']
developer: Goodwy
category: Photography
install: 10000
----------------------Loop i = 527----------------------
pkg_name: com.gallerypp.gallery.gallery
apk_purpose_general: Gallery
apk_purpose_detail: Gallery is organised your photos and videos and gallery album inside gallery app
['23.0 (23)', 'Jan 27, 2023', 'Gallery - Photo Album', 'Tools', 'com.gallerypp.gallery.gallery', '500,000+']
developer: Gallery - Photo Album
category: Tools
install: 500000
----------------------Loop i = 528----------------------
pkg_name: vit.nicegallery.iphoto
apk_purpose_general: iPhoto - Gallery iOS Style
apk_purpose_detail: iPhoto Gallery iOS Style is modern, fast photo gallery, picture manager app
['1.0.1 (1)', 'Apr 29, 202

----------------------Loop i = 547----------------------
pkg_name: com.prome.selfiecamera
apk_purpose_general: Selfie Camera: Beauty Camera, Photo Editor,Collage
apk_purpose_detail: Best Selfie Camera: Beauty Selfie Filters & Powerful Photo Editor & Fun sticker
['1.7.16.2 (60)', 'Dec 8, 2018', 'Fun Live', 'Photography', 'com.prome.selfiecamera', '100+']
developer: Fun Live
category: Photography
install: 100
----------------------Loop i = 548----------------------
pkg_name: com.meitu.meiyancamera
apk_purpose_general: BeautyCam - Beautify & AI Art
apk_purpose_detail: Generate your own AI portraits
['11.6.40 (11640)', 'Aug 24, 2023', 'Meitu (China) Limited', 'Photography', 'com.meitu.meiyancamera', '50,000,000+']
developer: Meitu (China) Limited
category: Photography
install: 50000000
----------------------Loop i = 549----------------------
pkg_name: com.galaxys.camera4k
apk_purpose_general: S22 Ultra Camera - Galaxy 4k
apk_purpose_detail: No need for an S22 phone to capture beautiful ima

----------------------Loop i = 567----------------------
pkg_name: com.album.photogallery.photomanager.gallerylock
apk_purpose_general: Gallery - Photo Gallery, Vault
apk_purpose_detail: Securely organize, view and protect your photos. Gallery app with built-in vault
['9.0 (9)', 'Aug 16, 2023', 'Gallery Photo LLC', 'Photography', 'com.album.photogallery.photomanager.gallerylock', '100,000+']
developer: Gallery Photo LLC
category: Photography
install: 100000
----------------------Loop i = 568----------------------
pkg_name: com.thinkyeah.galleryvault
apk_purpose_general: Gallery Vault-Hide Photo Video
apk_purpose_detail: Hide pictures & videos and download any videos; photo lock; cloud backup
['4.2.9 (2933)', 'Jul 11, 2023', 'GalleryVault Developer Team', 'Video Players & Editors', 'com.thinkyeah.galleryvault', '10,000,000+']
developer: GalleryVault Developer Team
category: Video Players & Editors
install: 10000000
----------------------Loop i = 569----------------------
pkg_name: galle

['3.51 (451)', 'Aug 7, 2023', 'Maxim Vasilkov', 'Tools', 'com.photo.vault.calculator', '10,000,000+']
developer: Maxim Vasilkov
category: Tools
install: 10000000
----------------------Loop i = 587----------------------
pkg_name: calculator.lock.calculatorlock
apk_purpose_general: Calculator Lock – HideZ
apk_purpose_detail: Hide Photos and Hide Videos, Perfect Image hider behind smart calculator
['1.0.4 (5)', 'Apr 9, 2022', 'TheByte.dev', 'Productivity', 'calculator.lock.calculatorlock', '100,000+']
developer: TheByte.dev
category: Productivity
install: 100000
----------------------Loop i = 588----------------------
pkg_name: com.istore.hide.app.calculator.vault.photo
apk_purpose_general: Calculator Lock: Gallery Vault
apk_purpose_detail: Calculator Lock – Photo & Video Vault allows you to hide all private items
['2.1 (19)', 'Nov 15, 2022', 'Braly JSC', 'Tools', 'com.istore.hide.app.calculator.vault.photo', '500,000+']
developer: Braly JSC
category: Tools
install: 500000
---------------

----------------------Loop i = 607----------------------
pkg_name: com.xiaomiCamera.redmik20pro
App does not exist
----------------------Loop i = 608----------------------
pkg_name: com.xiaomiCamera.redmi9c
App does not exist
----------------------Loop i = 609----------------------
pkg_name: com.VGANDEVart.CameraforREDMIk20
App does not exist
----------------------Loop i = 610----------------------
pkg_name: com.cameraformi.wallpaperformi.ringtoneformi.hdcamera
apk_purpose_general: Camera For Mi - Xiaomi Mi 12
apk_purpose_detail: Selfie Camera for Xiaomi Mi 12 Camera provides perfect quality for previewing
['1.0.4 (4)', 'Feb 27, 2023', 'Silver Softon', 'Photography', 'com.cameraformi.wallpaperformi.ringtoneformi.hdcamera', '10,000+']
developer: Silver Softon
category: Photography
install: 10000
----------------------Loop i = 611----------------------
pkg_name: com.firelolyopen.camera
apk_purpose_general: Open Camera
apk_purpose_detail: Generate your own AI portraits
['1.51.1 (87)', 'Au

In [148]:
exportResultCSV(df,apk_csv_file)